Try to find if trajectories feature can be used to support early stopping

In [ ]:
import os, sys
import json
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim

from scipy.spatial import distance
from scipy import stats

sys.path.append("..")
from singleVis.data import NormalDataProvider
from singleVis.eval.evaluator import Evaluator
from singleVis.projector import DVIProjector
from singleVis.vis_models import vis_models as vmodels

In [ ]:
VIS_METHOD = "tDVI" # DeepVisualInsight
# CONTENT_PATH = "/home/xianglin/projects/DVI_data/resnet18_cifar10"
# CONTENT_PATH = "/home/xianglin/projects/DVI_data/BadNet_MNIST_noise"
CONTENT_PATH = "/home/xianglin/projects/DVI_data/noisy/symmetric/cifar10/10"
GPU_ID = "0"

In [ ]:
sys.path.append(CONTENT_PATH)
with open(os.path.join(CONTENT_PATH, "config.json"), "r") as f:
    config = json.load(f)
config = config[VIS_METHOD]

In [ ]:
SETTING = config["SETTING"]
CLASSES = config["CLASSES"]
DATASET = config["DATASET"]
PREPROCESS = config["VISUALIZATION"]["PREPROCESS"]

# Training parameter (subject model)
TRAINING_PARAMETER = config["TRAINING"]
NET = TRAINING_PARAMETER["NET"]
LEN = TRAINING_PARAMETER["train_num"]
EPOCH_START = config["EPOCH_START"]
EPOCH_END = config["EPOCH_END"]
EPOCH_PERIOD = config["EPOCH_PERIOD"]
EPOCH_NAME = config["EPOCH_NAME"]

# Training parameter (visualization model)
VISUALIZATION_PARAMETER = config["VISUALIZATION"]
VIS_MODEL = VISUALIZATION_PARAMETER['VIS_MODEL']
LAMBDA = VISUALIZATION_PARAMETER["LAMBDA"]
B_N_EPOCHS = VISUALIZATION_PARAMETER["BOUNDARY"]["B_N_EPOCHS"]
L_BOUND = VISUALIZATION_PARAMETER["BOUNDARY"]["L_BOUND"]
ENCODER_DIMS = VISUALIZATION_PARAMETER["ENCODER_DIMS"]
DECODER_DIMS = VISUALIZATION_PARAMETER["DECODER_DIMS"]
S_N_EPOCHS = VISUALIZATION_PARAMETER["S_N_EPOCHS"]
N_NEIGHBORS = VISUALIZATION_PARAMETER["N_NEIGHBORS"]
PATIENT = VISUALIZATION_PARAMETER["PATIENT"]
MAX_EPOCH = VISUALIZATION_PARAMETER["MAX_EPOCH"]

# Define hyperparameters
DEVICE = torch.device("cuda:{}".format(GPU_ID) if torch.cuda.is_available() else "cpu")

In [ ]:
import Model.model as subject_model
net = eval("subject_model.{}()".format(NET))

In [ ]:
data_provider = NormalDataProvider(CONTENT_PATH, net, EPOCH_START, EPOCH_END, EPOCH_PERIOD, device=DEVICE, classes=CLASSES, epoch_name=EPOCH_NAME, verbose=1)

In [ ]:
VIS_MODEL_NAME = "tDVI_cnAE_estimation-sampling"
VIS_MODEL = "cnAE"
model = vmodels[VIS_MODEL](ENCODER_DIMS, DECODER_DIMS)
projector = DVIProjector(vis_model=model, content_path=CONTENT_PATH, vis_model_name=VIS_MODEL_NAME, epoch_name=EPOCH_NAME, device=DEVICE)
evaluator = Evaluator(data_provider,projector, metric="euclidean")

In [ ]:
import json
with open(f"{CONTENT_PATH}/noisy_label.json", "r") as f: 
    noise_labels = np.array(json.load(f))
with open(f"{CONTENT_PATH}/clean_label.json", "r") as f: 
    clean_labels = np.array(json.load(f))

idxs = np.argwhere(noise_labels!=clean_labels).squeeze()

In [ ]:
# idxs = np.array([])

In [ ]:
from singleVis.training_dynamics import TD
td = TD(data_provider, projector, (1,50,1))

In [ ]:
positions = td.position_high_dynamics()
td.show_ground_truth_cls(positions, idxs, 5, "UMAP")

In [ ]:
positions = td.position_dynamics()
td.show_ground_truth_cls(positions, idxs, 5, "RoG")

In [ ]:
losses = td.loss_dynamics()
td.show_ground_truth_cls(losses, idxs, 1, method="RoG")